# Получение и обработка статистических данных Канады

## Описание проекта:

Мой проект призван помочь ленивым людям (а также плохо владеющим английским языком), которым зачем-то потребовалось найти какие-то статистические данные Канады.
Суть проекта заключается в том, что, получая на вход запрос на русском или английском, переводит, если необходимо, запрос на английский с помощью сервиса Яндекс.Переводчик и ищет этот запрос на сайте статистикого сервиса Канада. Далее, по желанию, открываает первую таблицу данных среди соответствующих запросу и даже скачивает и разархивирует эту таблицу.
Также мой проект умеет строить графики по ВВП Канады за любой выбранный период времени, по любой индустрии.

В проекте использовались материалы с лекций, в некоторых местах практически копипастом.

In [78]:
#importing
import pandas as pd
import numpy as np
from selenium import webdriver
from selenium.webdriver.support.ui import WebDriverWait
from bs4 import BeautifulSoup
import re
import requests
import matplotlib.pyplot as plt
%matplotlib inline
from plotly.offline import init_notebook_mode, iplot
from plotly.graph_objs import Surface, Scatter3d
import zipfile
import os
import time
import sys
from selenium.common.exceptions import NoSuchElementException

In [79]:
#let's make normal text form that awful string which we got from yandex
def get_text(s):
    #в какой-то момент времени мне стало грустно от того, что ничего не получалось, поэтому этот кусок я реализовала несколькими способами :)
    list_splited=re.split('<|>', s) #способ 1
    str_res=''
    for i in range(0,len(list_splited), 2):
        str_res+=list_splited[i]
    
    #list_splited = [l_1[i].split('>') for i in range(len(l_1))]  #способ 2 через 
    
    #l_1 = s_1.split('<') #способ 3 - самый скучный
    #l_2=[]
    #for i in range(len(l_1)):
    #    l_2.append(l_1[i].split('>'))
    
    #str_res=''               #концовка для способа 2 и 3
    #for i in range(1,len(list_splited)-1):
    #    if len(list_splited[i])==2:
    #        str_res+=list_splited[i][1]
            
    return str_res

In [80]:
#translation with yandex
def translate(search='ввп'):
    #browser = webdriver.Chrome(executable_path=r'C:\Users\Anna\Downloads\data science\project\chromedriver.exe')
    ref = 'https://translate.yandex.ru/?lang=ru-en'
    browser.get(ref)

    element = browser.find_element_by_id("fakeArea")
    element.clear() # на случай, если это поле уже заполнено, очистим его
    element.send_keys(search)

    #browser.wait = (new WebDriverWait(browser, 1200)) #sometimes browser is too slow, so we should wait a bit to get the full translation
    #browser.implicitly_wait(60) 
    time.sleep(10)
    bs = BeautifulSoup(browser.page_source, "lxml")
    copy = bs.find_all("span", attrs={"data-complaint-type": "fullTextTranslation"})
    copy.append('')    
    return get_text(str(copy[0]))

In [81]:
#getting data about Canada from www150.statcan.gc.ca
def ca_find(find='gdp'):
    ref = 'https://www150.statcan.gc.ca/n1/en/type/data'
    browser.get(ref)
    element = browser.find_element_by_id("ndm-filter-text")
    element.clear() # на случай, если это поле уже заполнено, очистим его
    element.send_keys(find)
    element.submit()
    time.sleep(5)
    bs = BeautifulSoup(browser.page_source, "lxml")
    
    #ссылка на таблицу, которая оказалась первой в списке выдачи
    try:
        links = bs('details', id='tables')[0]  #tables нужно, чтобы открылись именно таблицы, а не что-то еще
        a=links('a')[0]
        a_string = str(a)
        index=a_string.find('href')
        string_for_help=a_string[index:len(a_string)]
        link=string_for_help.split('"')[1]
        return link
    except:
        return ''

In [82]:
#getting data about Canada from www150.statcan.gc.ca
def ca_pid(link='https://www150.statcan.gc.ca/t1/tbl1/en/tv.action?pid=3610043402'): #добываем PID если нас просили
    #getting PID from the link
    PID=link[link.rfind('pid'):len(link)]
    PID=PID.split('=')[1]
    if len(PID)>10:
        PID=PID[0:10]
    c=[]
    num= {'0','1','2','3','4','5','6','7','8','9'}
    for i in range(10):
        if not (PID[i] in num):
            c.append(i)
    if c!=[]:
        PID=PID[0:c[0]]
    return PID

In [83]:
#downloading search result
def ca_download(PID=36100434): #скачиваем файл, если нас просили
    #unfortunatelly sourse to which I desided to use API do not allow any parametrs except from product ID number which is their local system and I wasn't able to find dictionsry for it, but at least I can download file:
    #api: https://www.statcan.gc.ca/eng/developers/wds/user-guide
    if len(PID)>8:
        PID=PID[0:8]        
    url = "https://www150.statcan.gc.ca/t1/wds/rest/getFullTableDownloadCSV/"+ PID +"/en"
    get = requests.get(url)
    response = get.json()
    download_link=response['object']
    return download_link  

In [84]:
#unzipping files
#здесь использовалась информация с stackoverflow.com
def unzip(PID=36100434):
    if len(PID)>8:
        PID=PID[0:8]
    path2=os.getcwd()
    path = path2+'\\'+str(PID)+'-eng.zip'
    zipping = zipfile.ZipFile(path, 'r')
    zipping.extractall(path2)
    zipping.close()

In [85]:
#making a plot from canadian data
def plot_ca(start_time='2015-01', end_time='2019-01', customer_param = 1, industry='All industries', PID=36100434):
    df = pd.read_csv(path2+'//'+str(PID)+'.csv')
    df1=df[['REF_DATE', 'Seasonal adjustment', 'Prices', 'North American Industry Classification System (NAICS)', 'VALUE']]
    df2=df1.dropna()
    df2=df2.reset_index(drop=True)
    df3=df2[(df2['Seasonal adjustment']==param[customer_param][0])&(df2['Prices']==param[customer_param][1])&(df2['North American Industry Classification System (NAICS)']==industry)]
    df3=df3.set_index('REF_DATE')
    df3=df3[start_time:end_time]
    init_notebook_mode()
    iplot([{'x': df3.index, 'y': df3['VALUE'], 'mode': 'lines'}])

In [92]:
#main
param={0:['Seasonally adjusted at annual rates','Chained (2012) dollars'], 
            1:['Seasonally adjusted at annual rates','2012 constant prices'],
                     2:['Trading-day adjusted','2012 constant prices']}

print(' Please, select language. Reply "1" for Russian and "0" for English.\nПожалуйста, выберете язык. Ответьте "1", если Русский, и "0", если Английский.')
lang=int(input())
if lang:
    print(' Здравствуйте! Давайте начнем работу.\nЯ призван помочь вам найти данные о Канаде. Что вам хотелось бы найти сегодня?')#hi! let's get started!
    search = input() #what are we looking for
    print(' Я знаю, что вам, возможно лень общаться словами, так что давайте общаться цифрами! :) \nВы хотели бы скачать найденные данные?\n Ответьте "1", если да, и "0", если нет.')
    download=int(input())
    path2=os.getcwd()
    browser = webdriver.Chrome(executable_path=path2+'\chromedriver.exe')
    c=0 #проверяем, что запрос нужно переводить, так как хотя бы часть его на русском
    for i in range(len(search)):
        if 1039<ord(search[i])<1104:
            c+=1
    if c!=0:
        find=translate(search) #if nothing was returned -> 'Oops, looks like you have poor internet connection, please, check it and try again'
        if find=='':
            print(' Упс, ваш интернет слишком медленный для меня, у меня не получится ничего найти. Пожалуйста, попробуйте снова')
    else:
        find=search
    link=ca_find(find)
    if link=='':
        print(' Упс, кажется, по вашему запросу ничего нет :(')
    else:
        print(' Вот ссылка на первую базу данных, которую я нашел: '+link)
        PID=ca_pid(link)
        if len(PID)<8: #        "invalid PID"
            print(' Что-то пошло не так: номер базы меньше, чем должен быть. Ничего у нас с вами в это раз не получится. :(')
        else:
            print(' А вот номер этой базы данных в базе данных статистического центра: '+PID)
            download_link = ca_download(PID)
            if download:
                browser.get(download_link)
                print(' Вам разархировать скаченный файл?\n Ответьте "1", если да, и "0", если нет.')
                need_to_unzip=int(input())
                print(' Хорошо!')
                if need_to_unzip:
                    unzip(PID)
                    print(' Сделано!')
            else:
                print(' Если все же передумаете, то вот ссылка на скачивание файла: '+download_link)
            print(' У меня тут завалялись данные по ВВП Канады, хотите график построю?\nОтветьте "1", если да, и "0", если нет.')
            graph=int(input())
            if graph:
                print(' У меня есть данные с января 1997 года по январь 2019. Я хочу построить график с 2015 по 2019 год, вы хотели бы поменять время и вообще настроить график?\nОтветьте "1", если да, и "0", если нет.')
                costomize=int(input())
                if costomize:
                    print(' Введите, пожалуйста дату начала в формате "ГГГГ-ММ":')
                    start_time=input()
                    print(' Введите, пожалуйста дату окончания в формате "ГГГГ-ММ":')
                    end_time=input()
                    print(' Выберете формат данных: \n"0" - Сезонно скорректированные по годовым ставкам, в долларах 2012 года; \n"1" - Сезонно скорректированные по годовым ставкам, в ценах 2012 года, \n"2" - Скорректированные на торговый день, в ценах 2012 года')
                    customer_param=int(input())
                    #print('Выберете интересующую индустрию:')
                    #print(industry_list)
                    #industry=input()
                    #print('Введите номер продукта, если знаете') - на случай реализации построения и других графиков
                    #PID=int(input())
                    #plot_ca(start_time, end_time, customer_param, industry, PID)
                    plot_ca(start_time, end_time, customer_param)
                else:
                    plot_ca()
    print(' К сожалению, это пока все, что я умею. Надеюсь, я был полезен. До новых встреч!')

else:
    print(" Hello! Let's get started.\nI'm here to help you to find some data on Canada. What would you like to find today?")
    search = input() #what are we looking for
    print(" I know that you may be too lazy to communicate with words, so let's communicate with numbers! :)  \nWould you like to download the found data?\n Answer '1' if yes, and '0' if no.")
    download=int(input())
    path2=os.getcwd()
    browser = webdriver.Chrome(executable_path=path2+'\chromedriver.exe')
    c=0 #проверяем, что запрос нужно переводить, так как хотя бы часть его на русском
    for i in range(len(search)):
        if 1039<ord(search[i])<1104:
            c+=1
    if c!=0:
        find=translate(search) 
        if find=='':
            print("Oops, looks like you have poor internet connection, please, check it and try again")
    else:
        find=search
    link=ca_find(find)
    if link=='':
        print(" Oops, it seems to be nothing matching your request :(")
    else:
        print(' Here is a link to the first database I found: '+link)
        PID=ca_pid(link)
        if len(PID)<8: #        "invalid PID"
            print(' Something went wrong: the base number is less than it should be. Nothing will work this time. :(')
        else:
            print(' Here is product number of this database: '+PID)
            download_link = ca_download(PID)
            if download:
                browser.get(download_link)
                print(" Would you like to unzip the downloaded file?\n Answer '1' if yes, and '0' if no.")
                need_to_unzip=int(input())
                print(' Okey!')
                if need_to_unzip:
                    unzip(PID)
                    print(' Done!')
            else:
                print(' If you change your mind here is a link to download this file: '+download_link)
            print(" I have some data of GDP in Kanada. Would you like to get a plot?\n Answer '1' if yes, and '0' if no.")
            graph=int(input())
            if graph:
                print("  I have data from January 1997 to January 2019. I want to build a plot from 2015 to 2019, would you like to change the time selection and do any customization?\n Answer '1' if yes, and '0' if no.")
                costomize=int(input())
                if costomize:
                    print(' Please enter the starting date in the format "YYYY-MM":')
                    start_time=input()
                    print(' Please enter the ending date in the format "YYYY-MM":')
                    end_time=input()
                    print(' Please select data format: \n"0" - Seasonally adjusted at annual rates and chained (2012) dollars; \n"1" - Seasonally adjusted at annual rates and 2012 constant prices, \n"2" - Trading-day adjusted and 2012 constant prices')
                    customer_param=int(input())
                    #print('Выберете интересующую индустрию:') - на случай реализации построения с разными индустриями
                    #print(industry_list)
                    #industry=input()
                    #print('Введите номер продукта, если знаете') - на случай реализации построения и других графиков
                    #PID=int(input())
                    #plot_ca(start_time, end_time, customer_param, industry, PID)
                    plot_ca(start_time, end_time, customer_param)
                else:
                    plot_ca()
    print(" Unfortunately, that's all I know. I hope I've been helpful. See you next time!")            




 Please, select language. Reply "1" for Russian and "0" for English.
Пожалуйста, выберете язык. Ответьте "1", если Русский, и "0", если Английский.
0
 Hello! Let's get started.
I'm here to help you to find some data on Canada. What would you like to find today?
ввп
 I know that you may be too lazy to communicate with words, so let's communicate with numbers! :)  
Would you like to download the found data?
 Answer '1' if yes, and '0' if no.
1
 Here is a link to the first database I found: https://www150.statcan.gc.ca/t1/tbl1/en/tv.action?pid=3610043402
 Here is product number of this database: 3610043402
 Would you like to unzip the downloaded file?
 Answer '1' if yes, and '0' if no.
0
 Okey!
 I have some data of GDP in Kanada. Would you like to get a plot?
 Answer '1' if yes, and '0' if no.
0
 Unfortunately, that's all I know. I hope I've been helpful. See you next time!
